In [2]:
import sys
sys.path.append('../')
from settings import *
import pandas as pd
import networkx as nx
import matchmsextras.networking as net

In [3]:
data_path_net = '../data/processed/MolecularNetwork/'
cutoff=0.7 #Overwrite cutoff for MS2DeepScore

In [4]:
print('MS dataset: ', MSrawdata)
print('Similarity cut-off: ', cutoff)
print('Maxmimum edges per node: ', links)

MS dataset:  MassBank_NIST_Feb20
Similarity cut-off:  0.7
Maxmimum edges per node:  10


In [5]:
network = nx.read_graphml(data_path_net + MSrawdata+"_MS2DeepScore_cutoff"+str(cutoff)+"_links"+str(links)+".graphml")
tox=pd.read_csv('../data/processed/tox21/tox.csv')
tox.set_index('inchikey', inplace=True)
endpoints=list(tox.columns.values)

In [7]:
from sklearn.metrics import precision_recall_fscore_support
def calc_metrics (endpoint, network):
    
    predictions=pd.DataFrame(None)
    comparison=pd.DataFrame(None)
    predictions = pd.DataFrame(columns=['inchikey', 'predicted_activity'])

    for node in network.nodes:
        neighbors = list(network.neighbors(node))
        filtered_neighbors = [n for n in neighbors if not pd.isna(tox.loc[n].loc[endpoint])]
    
        if len(filtered_neighbors) == 0:
            predicted_activity = float("nan")
        else:
            num_zeros = sum(tox.loc[n].loc[endpoint] == 0 for n in filtered_neighbors)
            num_ones = len(filtered_neighbors) - num_zeros
    
            if num_zeros == num_ones:
                predicted_activity = 1
            else:
                if num_ones > num_zeros:
                    predicted_activity = 1
                else:
                    predicted_activity = 0
    
        temp_df = pd.DataFrame({'inchikey': [node], 'predicted_activity': [predicted_activity]})
        predictions = pd.concat([predictions, temp_df], ignore_index=True)
        
    predictions.set_index('inchikey', inplace=True)
    comparison=predictions.join(tox[endpoint]).dropna().astype('int')

    precision, recall, f1_score, support = precision_recall_fscore_support(comparison[endpoint], comparison['predicted_activity'], pos_label=1, average='binary')
    return endpoint, recall, precision, f1_score

In [8]:
columns = ['endpoint','recall','precision','F1-score']
rows = []
for endpoint in endpoints:
    rows.append(calc_metrics(endpoint, network))
df = pd.DataFrame(rows, columns=columns)
df

,endpoint,recall,precision,F1-score
0,NR.AhR,0.441558,0.304933,0.360743
1,NR.AR,0.714286,0.602410,0.653595
2,NR.AR.LBD,0.539683,0.557377,0.548387
3,NR.Aromatase,0.291667,0.148936,0.197183
4,NR.ER,0.403846,0.242775,0.303249
5,NR.ER.LBD,0.347826,0.197531,0.251969
6,NR.PPAR.gamma,0.071429,0.029412,0.041667
7,SR.ARE,0.422764,0.285714,0.340984
8,SR.ATAD5,0.333333,0.094340,0.147059
9,SR.HSE,0.055556,0.018519,0.027778
